In [2]:
!pip install xgboost
!pip install model_selection

In [9]:
import numpy as np
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.preprocessing import LabelEncoder
from google.cloud import storage, bigquery


In [17]:
client = bigquery.Client()
sql = "SELECT * FROM `bigquery-public-data.ml_datasets.iris`"
df = client.query(sql).to_dataframe()


In [21]:
target = df["species"]
features = df.drop("species", 1)
label_encoder = LabelEncoder()
target_encoder = label_encoder.fit_transform(target)
print(target_encoder)

[1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 1 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 2 1 1 2
 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 0 0 0 0 0
 0 0]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [26]:
# Split data
X_train,X_test, y_train, y_test = train_test_split(features, target_encoder, test_size=0.2, random_state=42)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [28]:
# train model
param = {
    'max_depth': 3
    , 'eta': 0.3
    , 'silent': 1
    , 'objective': 'multi:softprob'
    , 'num_class': 3
}
num_round = 10
bst = xgb.train(param, dtrain, num_round)

[05:12:21] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[05:12:21] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [31]:
#evaluate model
preds = bst.predict(dtest)
best_preds = np.asarray([np.argmax(line) for line in preds])
print(precision_score(y_test, best_preds, average='micro'))

0.9333333333333333


In [34]:
model = 'model.bst'
bst.save_model(model)
bucket = storage.Client().bucket('advde-bucket-1')
blob = bucket.blob('{}/{}'.format('ml/models/nb_model', model))
blob.upload_from_filename(model)